In [1]:
import utils
import predict
import time as tm
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.datasets import make_classification
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.functional as F

In [3]:
# from w2v import *
# from embedding_layer import embedding_layer
from sklearn import preprocessing
from sklearn.decomposition import PCA
import scipy.io as sio
from scipy import sparse
import argparse
from visdom import Visdom
from sklearn.externals import joblib 
# from futils import *
# from loss import loss

/home/aditya/Vpy35/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [4]:
# load Data
dictSize = 225
(X, y) = utils.loadData( "train", dictSize = dictSize )
X = scipy.sparse.csr_matrix.toarray(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
# https://discuss.pytorch.org/t/multi-class-classification/47565
# https://medium.com/dair-ai/a-simple-neural-network-from-scratch-with-pytorch-and-google-colab-c7f3830618e0
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out
    
    def predict(self, x):
        with torch.no_grad():
            outp = self.forward(x)
            return F.softmax(outp)
# add softmax layer: used for multiclass classification

In [6]:
# model declared
model = NeuralNet(225 , 1000, 51)  
# earlier hidden layers- 1000 
# 1 - poor, 2- ok

In [7]:
loss_function = nn.CrossEntropyLoss()
model_opt = optim.SGD(model.parameters(), lr = 0.02)

train_set_X = Variable(torch.from_numpy(X_train)).float()
train_set_y = Variable(torch.LongTensor(y_train)).long()
test_set_X = Variable(torch.from_numpy(X_test)).float()
test_set_y = Variable(torch.LongTensor(y_test)).long()

In [8]:
# 1
epochs = 50
for epochs in range(epochs):
    model_opt.zero_grad()
    out = model(train_set_X)
    loss = loss_function(out, train_set_y)
    loss.backward()
    model_opt.step()

In [9]:
ans = model.predict(test_set_X)
# for i in range(3299):
#     ans_max,ind = torch.max(ans[i],0)
#     print (test_set_y[i].numpy() - ind.numpy())
    

/home/aditya/Vpy35/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [10]:
k = 5
y_pr = ans.numpy() 
y_pred = np.argsort(-y_pr,axis=1)[:,:k]
print (y_pred)
print (y_test)

[[ 3  2  4  1  7]
 [ 2  1  3  4  9]
 [ 3  2  1  4 10]
 ...
 [ 2  1  3  4  9]
 [ 2  1  3  4  9]
 [ 3  2  1  4 10]]
[3. 2. 3. ... 9. 4. 3.]


In [11]:
# eval functions for Deep Learning
preck = utils.getPrecAtK( y_test, y_pred, k )
# The macro precision code takes a bit longer to execute due to the for loop over labels
mpreck = utils.getMPrecAtK( y_test, y_pred, k )

# According to our definitions, both prec@k and mprec@k should go up as k goes up i.e. for your
# method, prec@i > prec@j if i > j and mprec@i > mprec@j if i > j. See the assignment description
# to convince yourself why this must be the case.

print( "prec@1: %0.3f" % preck[0], "prec@3: %0.3f" % preck[2], "prec@5: %0.3f" % preck[4] )
# Dont be surprised if mprec is small -- it is hard to do well on rare error classes
print( "mprec@1: %0.3e" % mpreck[0], "mprec@3: %0.3e" % mpreck[2], "mprec@5: %0.3e" % mpreck[4] )
print ("prec matrix",preck)
print ("mprec matrix",mpreck)

prec@1: 0.314 prec@3: 0.614 prec@5: 0.752
mprec@1: 2.968e-02 mprec@3: 7.338e-02 mprec@5: 1.294e-01
prec matrix [0.31393939 0.50333333 0.61363636 0.69363636 0.75151515]
mprec matrix [0.02968053 0.05729217 0.07338063 0.08510638 0.12942324]


In [11]:
# 2 
# same model diff train file

In [12]:
tr_latent_X = data_utils.TensorDataset(train_set_X, train_set_y)
te_latent_X = data_utils.TensorDataset(test_set_X,  test_set_y)
train_loader_X = torch.utils.data.DataLoader(dataset=tr_latent_X)
test_loader_X = torch.utils.data.DataLoader(dataset=te_latent_X)

In [13]:
def train(epoch):
    model.train()

    train_loss = 0

    for batch_idx, (data,label) in enumerate(train_loader_X):

        model_opt.zero_grad()

        out = model(data)
        loss = loss_function(out, label)

        loss.backward()
        train_loss += loss.item()
        model_opt.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader_X.dataset),
                100. * batch_idx / len(train_loader_X), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader_X.dataset)))

In [14]:
# 2
for epoch in range(1, 10):
    train(epoch)

Train Epoch: 1 [0/6700 (0%)]	Loss: 2.854280
Train Epoch: 1 [100/6700 (1%)]	Loss: 2.127586
Train Epoch: 1 [200/6700 (3%)]	Loss: 0.779639
Train Epoch: 1 [300/6700 (4%)]	Loss: 4.261716
Train Epoch: 1 [400/6700 (6%)]	Loss: 3.735106
Train Epoch: 1 [500/6700 (7%)]	Loss: 1.373702
Train Epoch: 1 [600/6700 (9%)]	Loss: 1.796077
Train Epoch: 1 [700/6700 (10%)]	Loss: 0.799777
Train Epoch: 1 [800/6700 (12%)]	Loss: 2.275002
Train Epoch: 1 [900/6700 (13%)]	Loss: 0.553542
Train Epoch: 1 [1000/6700 (15%)]	Loss: 2.927905
Train Epoch: 1 [1100/6700 (16%)]	Loss: 0.007938
Train Epoch: 1 [1200/6700 (18%)]	Loss: 8.221107
Train Epoch: 1 [1300/6700 (19%)]	Loss: 0.170846
Train Epoch: 1 [1400/6700 (21%)]	Loss: 1.714790
Train Epoch: 1 [1500/6700 (22%)]	Loss: 0.666515
Train Epoch: 1 [1600/6700 (24%)]	Loss: 1.358858
Train Epoch: 1 [1700/6700 (25%)]	Loss: 0.041263
Train Epoch: 1 [1800/6700 (27%)]	Loss: 0.881191
Train Epoch: 1 [1900/6700 (28%)]	Loss: 1.189125
Train Epoch: 1 [2000/6700 (30%)]	Loss: 4.536890
Train Epoch

Train Epoch: 3 [3800/6700 (57%)]	Loss: 0.042151
Train Epoch: 3 [3900/6700 (58%)]	Loss: 0.048629
Train Epoch: 3 [4000/6700 (60%)]	Loss: 4.288632
Train Epoch: 3 [4100/6700 (61%)]	Loss: 0.258256
Train Epoch: 3 [4200/6700 (63%)]	Loss: 0.007191
Train Epoch: 3 [4300/6700 (64%)]	Loss: 0.958228
Train Epoch: 3 [4400/6700 (66%)]	Loss: 0.000575
Train Epoch: 3 [4500/6700 (67%)]	Loss: 2.134486
Train Epoch: 3 [4600/6700 (69%)]	Loss: 2.114346
Train Epoch: 3 [4700/6700 (70%)]	Loss: 0.196387
Train Epoch: 3 [4800/6700 (72%)]	Loss: 2.319604
Train Epoch: 3 [4900/6700 (73%)]	Loss: 0.394407
Train Epoch: 3 [5000/6700 (75%)]	Loss: 1.637612
Train Epoch: 3 [5100/6700 (76%)]	Loss: 0.151898
Train Epoch: 3 [5200/6700 (78%)]	Loss: 1.353289
Train Epoch: 3 [5300/6700 (79%)]	Loss: 0.622690
Train Epoch: 3 [5400/6700 (81%)]	Loss: 0.124146
Train Epoch: 3 [5500/6700 (82%)]	Loss: 2.925913
Train Epoch: 3 [5600/6700 (84%)]	Loss: 2.762436
Train Epoch: 3 [5700/6700 (85%)]	Loss: 2.358912
Train Epoch: 3 [5800/6700 (87%)]	Loss: 1

Train Epoch: 6 [800/6700 (12%)]	Loss: 1.106590
Train Epoch: 6 [900/6700 (13%)]	Loss: 0.081521
Train Epoch: 6 [1000/6700 (15%)]	Loss: 1.554601
Train Epoch: 6 [1100/6700 (16%)]	Loss: 0.081904
Train Epoch: 6 [1200/6700 (18%)]	Loss: 0.135601
Train Epoch: 6 [1300/6700 (19%)]	Loss: 0.025907
Train Epoch: 6 [1400/6700 (21%)]	Loss: 0.005191
Train Epoch: 6 [1500/6700 (22%)]	Loss: 0.109704
Train Epoch: 6 [1600/6700 (24%)]	Loss: 0.081467
Train Epoch: 6 [1700/6700 (25%)]	Loss: 0.002404
Train Epoch: 6 [1800/6700 (27%)]	Loss: 0.009346
Train Epoch: 6 [1900/6700 (28%)]	Loss: 0.407837
Train Epoch: 6 [2000/6700 (30%)]	Loss: 6.647480
Train Epoch: 6 [2100/6700 (31%)]	Loss: 0.030649
Train Epoch: 6 [2200/6700 (33%)]	Loss: 0.561600
Train Epoch: 6 [2300/6700 (34%)]	Loss: 0.005816
Train Epoch: 6 [2400/6700 (36%)]	Loss: 0.084271
Train Epoch: 6 [2500/6700 (37%)]	Loss: 0.051288
Train Epoch: 6 [2600/6700 (39%)]	Loss: 0.188711
Train Epoch: 6 [2700/6700 (40%)]	Loss: 0.426144
Train Epoch: 6 [2800/6700 (42%)]	Loss: 0.0

Train Epoch: 8 [4500/6700 (67%)]	Loss: 1.098346
Train Epoch: 8 [4600/6700 (69%)]	Loss: 0.378441
Train Epoch: 8 [4700/6700 (70%)]	Loss: 0.073670
Train Epoch: 8 [4800/6700 (72%)]	Loss: 2.694206
Train Epoch: 8 [4900/6700 (73%)]	Loss: 0.014123
Train Epoch: 8 [5000/6700 (75%)]	Loss: 0.558752
Train Epoch: 8 [5100/6700 (76%)]	Loss: 0.023359
Train Epoch: 8 [5200/6700 (78%)]	Loss: 0.283003
Train Epoch: 8 [5300/6700 (79%)]	Loss: 0.380264
Train Epoch: 8 [5400/6700 (81%)]	Loss: 0.117547
Train Epoch: 8 [5500/6700 (82%)]	Loss: 1.217327
Train Epoch: 8 [5600/6700 (84%)]	Loss: 0.971036
Train Epoch: 8 [5700/6700 (85%)]	Loss: 3.140064
Train Epoch: 8 [5800/6700 (87%)]	Loss: 1.176880
Train Epoch: 8 [5900/6700 (88%)]	Loss: 0.240503
Train Epoch: 8 [6000/6700 (90%)]	Loss: 0.178324
Train Epoch: 8 [6100/6700 (91%)]	Loss: 0.000245
Train Epoch: 8 [6200/6700 (93%)]	Loss: 0.005501
Train Epoch: 8 [6300/6700 (94%)]	Loss: 0.010676
Train Epoch: 8 [6400/6700 (96%)]	Loss: 0.037490
Train Epoch: 8 [6500/6700 (97%)]	Loss: 0

In [15]:
ans = model.predict(test_set_X)
k = 5
y_pr = ans.numpy() 
y_pred = np.argsort(-y_pr,axis=1)[:,:k]
print (y_pred)
print (y_test)

[[ 3 12 22  4  7]
 [ 2  1  9  4 15]
 [ 3 12  4 22 39]
 ...
 [ 9 45  1 15 16]
 [ 4 37  5 16 18]
 [ 3 12 22 37  4]]
[3. 2. 3. ... 9. 4. 3.]


/home/aditya/Vpy35/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [16]:
# eval functions for Deep Learning
preck = utils.getPrecAtK( y_test, y_pred, k )
# The macro precision code takes a bit longer to execute due to the for loop over labels
mpreck = utils.getMPrecAtK( y_test, y_pred, k )

# According to our definitions, both prec@k and mprec@k should go up as k goes up i.e. for your
# method, prec@i > prec@j if i > j and mprec@i > mprec@j if i > j. See the assignment description
# to convince yourself why this must be the case.

print( "prec@1: %0.3f" % preck[0], "prec@3: %0.3f" % preck[2], "prec@5: %0.3f" % preck[4] )
# Dont be surprised if mprec is small -- it is hard to do well on rare error classes
print( "mprec@1: %0.3e" % mpreck[0], "mprec@3: %0.3e" % mpreck[2], "mprec@5: %0.3e" % mpreck[4] )
print ("prec matrix",preck)
print ("mprec matrix",mpreck)

prec@1: 0.774 prec@3: 0.923 prec@5: 0.958
mprec@1: 5.077e-01 mprec@3: 7.707e-01 mprec@5: 8.564e-01
prec matrix [0.77363636 0.88878788 0.92272727 0.94363636 0.95787879]
mprec matrix [0.50767473 0.70322354 0.77065845 0.81476752 0.85638644]


In [59]:
# Improve nn 
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)
#         torch.nn.init.xavier_uniform_(self.layer1.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out
    
    def predict(self, x):
        with torch.no_grad():
            outp = self.forward(x)
            return F.softmax(outp)

In [60]:
# model declared
model = NeuralNet(225 , 2000, 51)

In [61]:
loss_function = nn.CrossEntropyLoss()
# loss_function = nn.BCELoss()
model_opt = optim.SGD(model.parameters(), lr = 2e-2)

train_set_X = Variable(torch.from_numpy(X_train)).float()
train_set_y = Variable(torch.LongTensor(y_train)).long()
test_set_X = Variable(torch.from_numpy(X_test)).float()
test_set_y = Variable(torch.LongTensor(y_test)).long()

tr_latent_X = data_utils.TensorDataset(train_set_X, train_set_y)
te_latent_X = data_utils.TensorDataset(test_set_X,  test_set_y)
train_loader_X = torch.utils.data.DataLoader(dataset=tr_latent_X)
test_loader_X = torch.utils.data.DataLoader(dataset=te_latent_X)

In [62]:
def train(epoch):
    model.train()

    train_loss = 0

    for batch_idx, (data,label) in enumerate(train_loader_X):

        model_opt.zero_grad()

        out = model(data)
        loss = loss_function(out, label)

        loss.backward()
        train_loss += loss.item()
        model_opt.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader_X.dataset),
                100. * batch_idx / len(train_loader_X), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader_X.dataset)))

In [63]:
for epoch in range(1, 10):
    train(epoch)

Train Epoch: 1 [0/6700 (0%)]	Loss: 4.035842
Train Epoch: 1 [100/6700 (1%)]	Loss: 2.159297
Train Epoch: 1 [200/6700 (3%)]	Loss: 0.764661
Train Epoch: 1 [300/6700 (4%)]	Loss: 4.318502
Train Epoch: 1 [400/6700 (6%)]	Loss: 3.904516
Train Epoch: 1 [500/6700 (7%)]	Loss: 1.309211
Train Epoch: 1 [600/6700 (9%)]	Loss: 1.763264
Train Epoch: 1 [700/6700 (10%)]	Loss: 0.840062
Train Epoch: 1 [800/6700 (12%)]	Loss: 2.210183
Train Epoch: 1 [900/6700 (13%)]	Loss: 0.477876
Train Epoch: 1 [1000/6700 (15%)]	Loss: 2.694440
Train Epoch: 1 [1100/6700 (16%)]	Loss: 0.010783
Train Epoch: 1 [1200/6700 (18%)]	Loss: 7.714949
Train Epoch: 1 [1300/6700 (19%)]	Loss: 0.160265
Train Epoch: 1 [1400/6700 (21%)]	Loss: 1.296306
Train Epoch: 1 [1500/6700 (22%)]	Loss: 0.595251
Train Epoch: 1 [1600/6700 (24%)]	Loss: 2.287751
Train Epoch: 1 [1700/6700 (25%)]	Loss: 0.043681
Train Epoch: 1 [1800/6700 (27%)]	Loss: 0.857507
Train Epoch: 1 [1900/6700 (28%)]	Loss: 1.402730
Train Epoch: 1 [2000/6700 (30%)]	Loss: 4.756148
Train Epoch

Train Epoch: 3 [3700/6700 (55%)]	Loss: 0.099102
Train Epoch: 3 [3800/6700 (57%)]	Loss: 0.045264
Train Epoch: 3 [3900/6700 (58%)]	Loss: 0.035081
Train Epoch: 3 [4000/6700 (60%)]	Loss: 3.370197
Train Epoch: 3 [4100/6700 (61%)]	Loss: 0.211939
Train Epoch: 3 [4200/6700 (63%)]	Loss: 0.006051
Train Epoch: 3 [4300/6700 (64%)]	Loss: 1.066134
Train Epoch: 3 [4400/6700 (66%)]	Loss: 0.006101
Train Epoch: 3 [4500/6700 (67%)]	Loss: 1.779349
Train Epoch: 3 [4600/6700 (69%)]	Loss: 1.661196
Train Epoch: 3 [4700/6700 (70%)]	Loss: 0.177092
Train Epoch: 3 [4800/6700 (72%)]	Loss: 2.329959
Train Epoch: 3 [4900/6700 (73%)]	Loss: 0.204493
Train Epoch: 3 [5000/6700 (75%)]	Loss: 1.851310
Train Epoch: 3 [5100/6700 (76%)]	Loss: 0.086548
Train Epoch: 3 [5200/6700 (78%)]	Loss: 1.382392
Train Epoch: 3 [5300/6700 (79%)]	Loss: 0.515427
Train Epoch: 3 [5400/6700 (81%)]	Loss: 0.148474
Train Epoch: 3 [5500/6700 (82%)]	Loss: 2.319941
Train Epoch: 3 [5600/6700 (84%)]	Loss: 2.818436
Train Epoch: 3 [5700/6700 (85%)]	Loss: 2

Train Epoch: 6 [600/6700 (9%)]	Loss: 0.177326
Train Epoch: 6 [700/6700 (10%)]	Loss: 0.323159
Train Epoch: 6 [800/6700 (12%)]	Loss: 0.818677
Train Epoch: 6 [900/6700 (13%)]	Loss: 0.082390
Train Epoch: 6 [1000/6700 (15%)]	Loss: 0.507445
Train Epoch: 6 [1100/6700 (16%)]	Loss: 0.049179
Train Epoch: 6 [1200/6700 (18%)]	Loss: 0.182306
Train Epoch: 6 [1300/6700 (19%)]	Loss: 0.020370
Train Epoch: 6 [1400/6700 (21%)]	Loss: 0.005878
Train Epoch: 6 [1500/6700 (22%)]	Loss: 0.096973
Train Epoch: 6 [1600/6700 (24%)]	Loss: 0.000310
Train Epoch: 6 [1700/6700 (25%)]	Loss: 0.001634
Train Epoch: 6 [1800/6700 (27%)]	Loss: 0.012320
Train Epoch: 6 [1900/6700 (28%)]	Loss: 0.427249
Train Epoch: 6 [2000/6700 (30%)]	Loss: 6.026429
Train Epoch: 6 [2100/6700 (31%)]	Loss: 0.027305
Train Epoch: 6 [2200/6700 (33%)]	Loss: 0.570731
Train Epoch: 6 [2300/6700 (34%)]	Loss: 0.004261
Train Epoch: 6 [2400/6700 (36%)]	Loss: 0.072330
Train Epoch: 6 [2500/6700 (37%)]	Loss: 0.050775
Train Epoch: 6 [2600/6700 (39%)]	Loss: 0.1614

Train Epoch: 8 [4300/6700 (64%)]	Loss: 0.771900
Train Epoch: 8 [4400/6700 (66%)]	Loss: 0.000346
Train Epoch: 8 [4500/6700 (67%)]	Loss: 0.441234
Train Epoch: 8 [4600/6700 (69%)]	Loss: 0.321379
Train Epoch: 8 [4700/6700 (70%)]	Loss: 0.059993
Train Epoch: 8 [4800/6700 (72%)]	Loss: 3.109362
Train Epoch: 8 [4900/6700 (73%)]	Loss: 0.007931
Train Epoch: 8 [5000/6700 (75%)]	Loss: 0.294278
Train Epoch: 8 [5100/6700 (76%)]	Loss: 0.019273
Train Epoch: 8 [5200/6700 (78%)]	Loss: 0.263948
Train Epoch: 8 [5300/6700 (79%)]	Loss: 0.365098
Train Epoch: 8 [5400/6700 (81%)]	Loss: 0.105974
Train Epoch: 8 [5500/6700 (82%)]	Loss: 0.162612
Train Epoch: 8 [5600/6700 (84%)]	Loss: 0.717741
Train Epoch: 8 [5700/6700 (85%)]	Loss: 2.886226
Train Epoch: 8 [5800/6700 (87%)]	Loss: 1.029425
Train Epoch: 8 [5900/6700 (88%)]	Loss: 0.152177
Train Epoch: 8 [6000/6700 (90%)]	Loss: 0.816631
Train Epoch: 8 [6100/6700 (91%)]	Loss: 0.000210
Train Epoch: 8 [6200/6700 (93%)]	Loss: 0.005838
Train Epoch: 8 [6300/6700 (94%)]	Loss: 0

In [64]:
ans = model.predict(test_set_X)
k = 5
y_pr = ans.numpy() 
y_pred = np.argsort(-y_pr,axis=1)[:,:k]
# eval functions for Deep Learning
preck = utils.getPrecAtK( y_test, y_pred, k )
# The macro precision code takes a bit longer to execute due to the for loop over labels
mpreck = utils.getMPrecAtK( y_test, y_pred, k )

# According to our definitions, both prec@k and mprec@k should go up as k goes up i.e. for your
# method, prec@i > prec@j if i > j and mprec@i > mprec@j if i > j. See the assignment description
# to convince yourself why this must be the case.

print( "prec@1: %0.3f" % preck[0], "prec@3: %0.3f" % preck[2], "prec@5: %0.3f" % preck[4] )
# Dont be surprised if mprec is small -- it is hard to do well on rare error classes
print( "mprec@1: %0.3e" % mpreck[0], "mprec@3: %0.3e" % mpreck[2], "mprec@5: %0.3e" % mpreck[4] )
print ("prec matrix",preck)
print ("mprec matrix",mpreck)

prec@1: 0.780 prec@3: 0.926 prec@5: 0.962
mprec@1: 5.306e-01 mprec@3: 7.742e-01 mprec@5: 8.665e-01
prec matrix [0.78       0.88818182 0.92606061 0.94606061 0.96151515]
mprec matrix [0.53062151 0.69080002 0.77422045 0.82144886 0.86652688]


/home/aditya/Vpy35/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
